In [1]:
import collections

# NOTES

### general notes
1. problems denoted by * proved to be quite challenging for me upon first attempt
2. problems denoted by ** are exceedingly challenging and will require a lot of revision
3. problems denoted by @ remain incomplete
4. in leetcode, problems with notes in green are first attempts/solutions or reworks after failure. those with notes in blue are review

### study notes

# 5b. Backtracking

# 6a. Heap / Priority Queue

# 6c. 1-D Dynamic Programming